In [ ]:
pip install -U langchain
# Requires Python 3.10+
# Installing the OpenAI integration
pip install -U langchain-openai

# 构建 Agent 的一些核心要素
1. 系统提示词：Detailed system prompts for better agent behavior
2. 工具函数：Create tools that integrate with external data
3. 模型配置：Model configuration for consistent response1s
4. 标准化输出：Structured output for predictable results
5. 会话记忆： Conversational memory for chat-like interactions
6. 创建并运行：Create and run the agent create a fully functional agent

# 接下来：        
- Build a basic agent：一个简单的例子
- Build a real-world agent：更接近真实的例子

# 1 Build a basic agent

In [28]:
# 一个超级简单的Agent
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = "5a4a5c93bdc34d8caac98deac76ecc82.uQpxDjyCez9svVjW"

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

def des_animal(animal: str) -> str:
    """Describe an animal."""
    return f"{animal} are very cute animals."

llm = ChatOpenAI(
    model = "glm-4.6",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/",
    )

agent = create_agent(
    model=llm,
    tools=[get_weather, des_animal],
    system_prompt="You are a helpful assistant",
)

# Run the agent
response1 = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)
response2 = agent.invoke(
    {"messages": [{"role": "user", "content": "describe panda"}]}
)

## 查看 返回值 的类型

In [33]:
print(type(response1),type(response1['messages']), response1.keys(),len(response1['messages']),'\n', 
      type(response1['messages'][0]), '\n',
      type(response1['messages'][1]), '\n',
      type(response1['messages'][2]), '\n',
      type(response1['messages'][3]))

<class 'dict'> <class 'list'> dict_keys(['messages']) 4 
 <class 'langchain_core.messages.human.HumanMessage'> 
 <class 'langchain_core.messages.ai.AIMessage'> 
 <class 'langchain_core.messages.tool.ToolMessage'> 
 <class 'langchain_core.messages.ai.AIMessage'>


In [ ]:
print(response1['messages'][0].content, '\n---\n',
        response1['messages'][1].content, '\n---\n',
        response1['messages'][2].content, '\n---\n',
        response1['messages'][3].content
      )

what is the weather in sf 
---
 
I'll check the weather in San Francisco for you.
 
---
 It's always sunny in San Francisco! 
---
 
The weather function indicates that "It's always sunny in San Francisco!" 

However, this seems to be a generic response rather than actual current weather data. For real-time weather information in San Francisco, you might want to check a reliable weather service or app for the most accurate and up-to-date conditions, including temperature, humidity, wind speed, and any weather alerts.


In [ ]:
print(response2['messages'][0].content, '\n---\n',
      response2['messages'][1].content, '\n---\n',
      response2['messages'][2],response2['messages'][2].content, '\n---\n',
      response2['messages'][3].content
      )

describe panda 
---
 
 
---
 panda are very cute animals. 
---
 
Pandas are very cute animals! They are known for their distinctive black and white coloring, with black patches around their eyes, ears, and across their round bodies. These gentle giants are native to China and primarily eat bamboo - they can consume 20-40 pounds of bamboo per day! 

Pandas are known for their playful and sometimes clumsy behavior, and they spend most of their time eating or sleeping. They're solitary animals in the wild but are beloved around the world as symbols of wildlife conservation. Unfortunately, they are classified as vulnerable due to habitat loss, but conservation efforts have helped their populations slowly recover.


# 2 Build a real-world agent
依据六个核心要素 逐步实现

### 1. Define the SYSTEM_PROMPT

In [ ]:
# 描述我们实现了两个 tools：get_weather_for_location 和 get_user_location
# 以此告诉模型在用户询问天气时要确保知道位置，如果不确定位置，就调用 get_user_location 工具来获取用户的位置。
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

### 2. Create tools

In [ ]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

# 这个 @tool 装饰器 会将普通函数 转换为 tool
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:

    #Tools should be well-documented: their name, description, and argument names become part of the model’s prompt. 
    # LangChain’s @tool decorator adds metadata and enables runtime injection via the ToolRuntime parameter.

    # 针对这个 ToolRuntime，我们定义了一个 Context 数据类 来描述上下文结构，目前看出来的作用是简化参数的传递，方便管理参数。

    """Retrieve user information based on user ID."""
    
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

### 3 Configure your model


In [ ]:

"""
加载模型与参数，GLM系列模型加载参考上文
一般来说，有四种方式加载模型：
1. 使用 langchain_openai 库中的 ChatOpenAI 类
2. OpenAI API 直接调用或兼容调用
3. Claude API 直接调用或兼容调用
4. 模型本身 的 官方 SDK
"""

### 4 Define response format

In [ ]:
# 可选的结构化输出。Optionally, define a structured response format if you need the agent responses to match a specific schema.
# 通过 dataclasses 装饰器来实现

from dataclasses import dataclass

# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

### 5 Add memory 

In [ ]:

# 记忆模块，生产中，使用一个持久化的 checkpointer 将数据存到数据库
# In production, use a persistent checkpointer that saves to a database.
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

### 6 Create and run the agent

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
# 取名怪怪的，区分不同的对话
config = {"configurable": {"thread_id": "1"}}

# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)
print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# 完整代码

In [ ]:
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver


# Define system prompt
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

# Define context schema
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

# Define tools
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

# Configure model
model = init_chat_model(
    "claude-sonnet-4-5-20250929",
    temperature=0
)

# Define response format
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

# Set up memory
checkpointer = InMemorySaver()

# Create agent
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

# Run agent
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )